In [3]:
import telebot
import requests
from bs4 import BeautifulSoup
import openpyxl

bot = telebot.TeleBot('AAEV_lK5gloGpAPiQRpfbS42cgYZfm2xjmo')


def get_news_from_channels(chat_id, channels):
    # Создаем новый Excel-файл
    wb = openpyxl.Workbook()
    sheet = wb.active

    # Для каждого канала пользователя
    for channel in channels:
        # Получаем последние 10 сообщений
        url = f'https://t.me/{channel}?before=1'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        messages = soup.find_all('div', {'class': 'tgme_widget_message_bubble'})

        # Добавляем заголовки в таблицу
        sheet.append([f'Новости из канала {channel}'])
        sheet.append(['Заголовок', 'Описание'])

        # Добавляем новости в таблицу
        for message in messages:
            title = message.find('div', {'class': 'tgme_widget_message_text'}).text.strip()
            description = message.find('div', {'class': 'tgme_widget_message_text'}).find_next('div').text.strip()
            sheet.append([title, description])

    # Сохраняем файл и отправляем его пользователю
    wb.save('news.xlsx')
    with open('news.xlsx', 'rb') as file:
        bot.send_document(chat_id, file)


@bot.message_handler(commands=['get_news'])
def handle_get_news(message):
    # Запрашиваем у пользователя список каналов
    bot.send_message(message.chat.id, 'Введите список каналов через запятую:')
    bot.register_next_step_handler(message, lambda m: get_news_from_channels(m.chat.id, m.text.split(',')))

